In [1]:
%%cmd
pip install shapely
pip install pyproj
pip install pyogrio
pip install geopandas[all]

Microsoft Windows [Version 10.0.22631.4890]
(c) Microsoft Corporation. All rights reserved.

C:\Users\vivgo\OneDrive\Documents\Assignments Spring 2025\HDS5230>pip install shapely

C:\Users\vivgo\OneDrive\Documents\Assignments Spring 2025\HDS5230>pip install pyproj

C:\Users\vivgo\OneDrive\Documents\Assignments Spring 2025\HDS5230>pip install pyogrio

C:\Users\vivgo\OneDrive\Documents\Assignments Spring 2025\HDS5230>pip install geopandas[all]

C:\Users\vivgo\OneDrive\Documents\Assignments Spring 2025\HDS5230>

In [407]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
import random
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
import math
from math import sqrt
import copy
from functools import *
import time
import itertools

In [3]:
# Read the shapefile
shapefile_path = "MO_2018_Federally_Qualified_Health_Center_Locations"  # Replace with the actual path
gdf = gpd.read_file(shapefile_path)

In [4]:
print(gdf.crs) # print the coordinate reference system
crs = gdf.crs

EPSG:4326


In [5]:
gdf.head()

,OBJECTID,Group_Name,Facility,Address,City,County,State,Zip,Phone,Latitude,Longitude,Loc_Code,geometry
0,1,COMTREA,COMTREA Byrnes Mill Health Center,100 Osage Executive Circle,House Springs,Jefferson,MO,63051,6367893372,38.435946,-90.554678,MAP,POINT (-90.55472 38.43597)
1,2,Missouri Highlands Health Care,Viburnum Medical Clinic,18 Viburnum Center Road,Viburnum,Iron,MO,65566,5732445406,37.714620,-91.133983,MAP,POINT (-91.13403 37.71462)
2,3,Central Ozarks Medical Center,Central Ozarks Medical Center At The Lake,3870 Columbia Avenue,Osage Beach,Miller,MO,65065,5733027490,38.160258,-92.601463,MAP,POINT (-92.60144 38.16025)
3,4,Missouri Highlands Health Care,Missouri Highland Medical Clinic - Poplar Bluf...,"225 Physicians Park Drive, Suite 303",Poplar Bluff,Butler,MO,63901,5737856536,36.772568,-90.457206,MAP,POINT (-90.45724 36.77261)
4,5,Swope Health Services,Swope Health Hickman Mills,8800 Blue Ridge Boulevard,Kansas City,Jackson,MO,64138,8163213200,38.962882,-94.498847,MAP,POINT (-94.49886 38.9629)


In [6]:
gdf = gdf.set_index("OBJECTID")

In [7]:
def create_point_column(df, lon_col, lat_col, crs="EPSG:4326"):
    """
    Creates a geometry column of Point objects from longitude and latitude columns in a DataFrame.

    Args:
        df (pandas.DataFrame): The DataFrame containing longitude and latitude columns.
        lon_col (str): The name of the longitude column.
        lat_col (str): The name of the latitude column.
        crs (str, optional): Coordinate Reference System. Defaults to "EPSG:4326".

    Returns:
        geopandas.GeoDataFrame: A GeoDataFrame with the added geometry column.
    """

    geometry = [Point(xy) for xy in zip(df[lon_col], df[lat_col])]
    gdf = gpd.GeoDataFrame(df, geometry=geometry, crs=crs)
    return gdf

In [372]:
# Read the shapefile
pop_file_path = "Mo_pop_sim/Mo_pop_Sim.csv"  # Replace with the actual path
big_pop_df = pd.read_csv(pop_file_path)
pop_df = big_pop_df.sample(frac=0.00005) 

pop_gdf = create_point_column(pop_df, 'long', 'lat', crs)
print(len(pop_gdf))

317


In [426]:
def fitness(fqhc_subset, pop_gdf):
    """
    Compute the average distance from residents to their nearest FQHC in the given subset.

    Args:
        fqhc_subset (GeoDataFrame): A subset of 8 selected FQHCs.
        pop_gdf (GeoDataFrame): The resident locations.

    Returns:
        float: The average distance between residents and their nearest FQHC.
    """
    # Ensure both datasets are in a projected CRS
    if pop_gdf.crs.is_geographic:
        pop_gdf = pop_gdf.to_crs(epsg=26996)  # Replace with your local projected CRS
    
    if fqhc_subset.crs.is_geographic:
        fqhc_subset = fqhc_subset.to_crs(epsg=26996)  # Same projection

    # Perform nearest spatial join
    nearest_fqhc = gpd.sjoin_nearest(pop_gdf, fqhc_subset, how="left")

    # Ensure correct column naming for FQHC geometry
    nearest_fqhc = nearest_fqhc.merge(fqhc_subset[['geometry']], left_on='OBJECTID', right_index=True, suffixes=('_pop', '_fqhc'))

    # Compute the average distance
    avg_distance = nearest_fqhc['geometry_pop'].distance(nearest_fqhc['geometry_fqhc']).mean()

    #getting distance penalty
    distance_penalty=0
    distance_penalty = max(0, avg_distance - 11000)
    
    fitness_value = (avg_distance + distance_penalty) / 10000
    return fitness_value

In [427]:
def initialize_population(gdf, pop_size):
    """
    Generate an initial population of random FQHC selections.
    
    Args:
        gdf (GeoDataFrame): Full list of FQHC locations.
        pop_size (int): Number of individuals (solutions) in the population.

    Returns:
        list: A list of subsets (each with 8 FQHCs).
    """
    population = []
    for _ in range(pop_size):
        individual = gdf.sample(n=8)  # Randomly select 8 FQHCs
        population.append(individual)
        
    return population

In [433]:
def perform_selection(population, pop_gdf, tournament_size=6):
    """
    Select parents using tournament selection.

    Args:
        population (list): List of FQHC subsets.
        pop_gdf (GeoDataFrame): Residents data.
        tournament_size (int): Number of individuals selected for each tournament.

    Returns:
        list: Two selected parents.
    """
    # Calculate the fitness of each individual
    fitness_values = [fitness(individual, pop_gdf) for individual in population]
    
    parents = []
    for _ in range(2):
        # Randomly select individuals for the tournament
        tournament = random.sample(list(zip(population, fitness_values)), tournament_size)
        
        # Find the individual with the best fitness (lowest fitness value)
        best_individual = min(tournament, key=lambda x: x[1])  # Assuming lower fitness is better
        parents.append(best_individual[0])  # Add the best individual to the parents list

    return parents


In [434]:
def crossover(parent1, parent2, crossover_rate = 0.85):
    """
    Perform crossover by swapping some FQHCs between parents.
    
    Args:
        parent1 (GeoDataFrame): First parent.
        parent2 (GeoDataFrame): Second parent.
        gdf (GeoDataFrame): Full dataset of FQHCs to sample from.
        crossover_rate (float): Probability of performing crossover.

    Returns:
        GeoDataFrame: A new child solution.
    """
    if random.random() < crossover_rate:
        crossover_point = random.randint(1, len(parent1) - 1)  # Random split point
        child_fqhc = pd.concat([parent1.iloc[:crossover_point], parent2.iloc[crossover_point:]]).drop_duplicates()

        # Ensure exactly 8 FQHCs
        available_fqhc = gdf[~gdf.index.isin(child_fqhc.index)]
        while len(child_fqhc) < 8 and not available_fqhc.empty:
            additional_fqhc = available_fqhc.sample(1)
            child_fqhc = pd.concat([child_fqhc, additional_fqhc]).drop_duplicates()
            available_fqhc = available_fqhc[~available_fqhc.index.isin(child_fqhc.index)]  # Update available pool

    else:
        child_fqhc = parent1.copy()  # No crossover, clone one parent
    
    return child_fqhc

In [435]:
def mutate(individual, mutation_rate=0.45):
    """
    Randomly swap two FQHCs with new ones.

    Args:
        individual (GeoDataFrame): A subset of 8 FQHCs.
        gdf (GeoDataFrame): Full dataset of FQHCs to sample from.
        mutation_rate (float): Probability of mutation.

    Returns:
        GeoDataFrame: Mutated FQHC selection.
    """
    if random.random() < mutation_rate:
        mutated_individual = individual.copy()

        # Pick two distinct indices to replace
        replace_idxs = random.sample(range(8), 2)
        
        # Get available FQHCs (those not already in the individual)
        available_fqhc = gdf[~gdf.index.isin(mutated_individual.index)]
        
        if len(available_fqhc) >= 2:  # Ensure we have enough to swap
            new_fqhc = available_fqhc.sample(n=2)  # Pick two new random FQHCs
            mutated_individual.iloc[replace_idxs[0]] = new_fqhc.iloc[0]
            mutated_individual.iloc[replace_idxs[1]] = new_fqhc.iloc[1]

        return mutated_individual
    
    return individual  # Return unchanged if mutation doesn't occur

In [436]:
def run_algorithm(gdf, pop_gdf, generations=50, pop_size=100, patience=5, reset_threshold=7, elitism_rate=1):
    """
    Run the genetic algorithm to find the best 8 FQHCs with early stopping.
    
    Args:
        gdf (GeoDataFrame): The full list of FQHCs.
        pop_gdf (GeoDataFrame): The resident locations.
        generations (int): Number of generations to run.
        pop_size (int): Number of individuals in each generation.
        patience (int): Number of generations without improvement before stopping early.

    Returns:
        GeoDataFrame: The best set of 8 FQHCs found.
    """
    population = initialize_population(gdf, pop_size)
    best_fitness = float('inf')  # Initialize with a high value
    stagnant_generations = 0  # Count generations without improvement

    for generation in range(generations):
        new_population = []

        # Select and retain elitism
        elites = sorted(population, key=lambda x: fitness(x, pop_gdf))[:elitism_rate]
        new_population.extend(elites)  # Start the new population with elites
        
        # Generate new offspring by performing selection, crossover, and mutation
        for _ in range(pop_size-elitism_rate):  
            parent1, parent2 = perform_selection(population, pop_gdf)
            child = crossover(parent1, parent2)
            child = mutate(child)  # Mutate the child
            new_population.append(child)

        assert len(new_population) == pop_size, "Population size mismatch!"
        population = new_population  # Update population
        
        # Track the best solution in this generation
        best_individual = min(population, key=lambda x: fitness(x, pop_gdf))
        best_distance = fitness(best_individual, pop_gdf)

        print(f"Generation {generation+1}: Best Avg Distance = {best_distance}")

        # Check for improvement
        if best_distance < best_fitness:
            best_fitness = best_distance
            stagnant_generations = 0  # Reset counter
        else:
            stagnant_generations += 1  # Increase counter

        if stagnant_generations >= reset_threshold:
            print(f"Population reset at Generation {generation+1}: No improvement for {reset_threshold} generations.")

            # **Keep the elite individual**
            new_population = [best_individual]  

            # **Reinitialize the rest of the population**
            new_population.extend(initialize_population(gdf, pop_size - 1))
            
            # **Reset stagnation counter**
            stagnant_generations = 0  

            # Update population
            population = new_population
            
        # Stop if no improvement for `patience` generations
        #if stagnant_generations >= patience:
            #print(f"Early stopping at Generation {generation+1}: No improvement for {patience} generations.")
            #break
    
    return best_individual  # Return best found solution


In [ ]:
best_fqhc_set = run_algorithm(gdf, pop_gdf)
best_fqhc_set

Generation 1: Best Avg Distance = 7.765806701994623
Generation 2: Best Avg Distance = 7.326880590960503
Generation 3: Best Avg Distance = 7.326880590960503
Generation 4: Best Avg Distance = 7.326880590960503
Generation 5: Best Avg Distance = 7.326880590960503
Generation 6: Best Avg Distance = 7.326880590960503
Generation 7: Best Avg Distance = 7.182725561614231
Generation 8: Best Avg Distance = 7.150604452277992
Generation 9: Best Avg Distance = 7.132780358211151
Generation 10: Best Avg Distance = 7.0808663928951425
Generation 11: Best Avg Distance = 6.998266119697016
Generation 12: Best Avg Distance = 6.895389379796799
Generation 13: Best Avg Distance = 6.824126837580056
Generation 14: Best Avg Distance = 6.824126837580056
Generation 15: Best Avg Distance = 6.733445334832861
Generation 16: Best Avg Distance = 6.733445334832861
Generation 17: Best Avg Distance = 6.733445334832861
Generation 18: Best Avg Distance = 6.733445334832861
Generation 19: Best Avg Distance = 6.733445334832861
G

In [373]:
def test_minimum_mean_distance(gdf, pop_gdf):
    """
    Calculate the absolute minimum of mean distance from residents to their nearest FQHC.
    
    Args:
        gdf (GeoDataFrame): FQHC locations GeoDataFrame.
        pop_gdf (GeoDataFrame): Resident locations GeoDataFrame.
    
    Returns:
        float: The absolute minimum mean distance.
    """
    # Reproject to the same CRS
    if pop_gdf.crs.is_geographic:
        pop_gdf = pop_gdf.to_crs(epsg=26996)
    if gdf.crs.is_geographic:
        gdf = gdf.to_crs(epsg=26996)

    # Perform nearest spatial join to find the nearest FQHC for each resident
    nearest_fqhc = gpd.sjoin_nearest(pop_gdf, gdf, how="left")

    # Compute the distance between residents and nearest FQHC using the geometry column
    nearest_fqhc = nearest_fqhc.merge(gdf[['geometry']], left_on='OBJECTID', right_index=True, suffixes=('_pop', '_fqhc'))

    # Compute the average distance
    avg_distance = nearest_fqhc['geometry_pop'].distance(nearest_fqhc['geometry_fqhc']).mean()
    
    return avg_distance


In [374]:
min_distance = test_minimum_mean_distance(gdf, pop_gdf)
print(f"Minimum Mean Distance: {min_distance}")

Minimum Mean Distance: 11450.664643262175
